# Proyecto
______________

API: https://dev.adalab.es/cinema/
Endpoint : https://dev.adalab.es/api/cinema/movies?year=2010&genre=Drama

- **Librerías**

In [50]:
#Importar librerías para su posterior uso en el código
import tqdm
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import datetime
import json
from time import sleep
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
# Importar librerías de Selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

# Comprobando que el display sea el máximo para mostrar mejor el data frame
pd.set_option('display.max_rows', 100)  # Cambia el número máximo de filas mostradas
pd.set_option('display.max_columns', 10)  # Cambia el número máximo de columnas mostradas
pd.set_option('display.width', 1000)  # Cambia el ancho máximo de la pantalla

- **Extracción API** (generos y rango años 15)

In [51]:

# Lista de años buscados: 15 años
anio_lista = list(range(2009, 2025))

# Lista de géneros
generos_lista = ["Drama", "Comedy", "Action", "Fantasy", "Horror", "Mystery", "Romance", "Thriller"]

# Lista vacía en la cual almacenaremos los datos extrañidos de la API
datos_peliculas = []

# Bucle para iterar por año y género
for anio in anio_lista:
    for genero in generos_lista:
        # Determinar URL
        url_movies = f"https://dev.adalab.es/api/cinema/movies?year={anio}&genre={genero}"
        
        # Llamada a API
        res_url_movies = requests.get(url_movies)
        
        # Pausa para la API
        sleep(1) 
        
        # Verificar la espuesta de la API
        if res_url_movies.status_code == 200:
            try:
                json_url_general = res_url_movies.json()
                
                # Imprimir jason para verlo
                print(f"Response for {anio} - {genero}: {json_url_general}")
                
                # Verificar si la llamada tiene la info correcta
                if isinstance(json_url_general, dict) and 'results' in json_url_general:
                    movies = json_url_general['results']
                    
                    # Verificar que es lista
                    if isinstance(movies, list):
                        # Bucle por película para extraer info
                        for pelicula in movies:
                            id = pelicula.get("idOwner")
                            titulo = pelicula.get("title")
                            tipo = pelicula.get("type")
                            anio = pelicula.get("year")
                            genero = pelicula.get("genre")
                            
                            # Añadir info a la lista anteriormente vacía
                            datos_peliculas.append((id, titulo, tipo, anio, genero))
                    else:
                        print("Expected 'results' to be a list.")
                else:
                    print("Expected JSON structure not found.")
            except json.JSONDecodeError:
                print("Error decoding JSON response.")
        else:
            print(f"Error: {res_url_movies.status_code} - {res_url_movies.reason}")

# Convertir a pandas frame
df_peliculas = pd.DataFrame(datos_peliculas)

# Salvar la busqueda en un documento con formato CSV
df_peliculas.to_csv("movies_last_15_years.csv", index=False)

print("Data extraction complete and saved to movies_last_15_years.csv.")


Response for 2009 - Drama: {'info': {'count': 10}, 'results': [{'id': 59404, 'title': 'Apaföld', 'type': 'Movie', 'year': '2009', 'genre': 'Drama', 'idOwner': 'tt0143558', 'image': 'https://m.media-amazon.com/images/M/MV5BYjA5Y2M3ZDAtNjg4OC00ZDQ5LWI4MzgtM2E0NTdjZmUwZjE1XkEyXkFqcGdeQXVyNDYzMDMxOTc@._V1_.jpg'}, {'id': 59405, 'title': 'Rózsaszín sajt', 'type': 'Movie', 'year': '2009', 'genre': 'Drama', 'idOwner': 'tt0153140', 'image': 'https://m.media-amazon.com/images/M/MV5BMjE2NDAxNDE0Nl5BMl5BanBnXkFtZTgwMTU1MjQwNjE@._V1_.jpg'}, {'id': 59406, 'title': 'Sanam Teri Kasam', 'type': 'Movie', 'year': '2009', 'genre': 'Drama', 'idOwner': 'tt0205380', 'image': 'https://m.media-amazon.com/images/M/MV5BYjc3MGNhMmItMjljOS00NjY5LThlMmQtYWM1NWZiOGJlY2EyXkEyXkFqcGdeQXVyMjU4NDY1ODA@._V1_.jpg'}, {'id': 59407, 'title': 'Twice as Dead', 'type': 'Movie', 'year': '2009', 'genre': 'Drama', 'idOwner': 'tt0254832', 'image': 'https://m.media-amazon.com/images/M/MV5BMTI2NjA5NDkyMl5BMl5BanBnXkFtZTcwNjAxNTEwMw@@

In [52]:
print(datos_peliculas)
print(type(datos_peliculas))

[('tt0143558', 'Apaföld', 'Movie', '2009', 'Drama'), ('tt0153140', 'Rózsaszín sajt', 'Movie', '2009', 'Drama'), ('tt0205380', 'Sanam Teri Kasam', 'Movie', '2009', 'Drama'), ('tt0254832', 'Twice as Dead', 'Movie', '2009', 'Drama'), ('tt0311698', 'Railed', 'Movie', '2009', 'Drama'), ('tt0327597', 'Coraline', 'Movie', '2009', 'Drama'), ('tt0344074', 'Mitsein', 'Movie', '2009', 'Drama'), ('tt0346631', 'Blood and Bone', 'Movie', '2009', 'Drama'), ('tt0361748', 'Inglourious Basterds', 'Movie', '2009', 'Drama'), ('tt0362478', 'The Box', 'Movie', '2009', 'Drama'), ('tt0067230', 'I Miss Sonia Henie', 'Short', '2009', 'Comedy'), ('tt0288190', 'El reino de los cielos', 'Movie', '2009', 'Comedy'), ('tt0304876', 'Unter Strom', 'Movie', '2009', 'Comedy'), ('tt0312958', 'Chinango', 'Movie', '2009', 'Comedy'), ('tt0345683', 'Mutant Swinger from Mars', 'Video', '2009', 'Comedy'), ('tt0353765', 'Los muertos van deprisa', 'Movie', '2009', 'Comedy'), ('tt0365873', 'Two-Minute Heist', 'Movie', '2009', 'Com

In [53]:
# Obtiene los nombres en la posición 2 (0,1,2) que es en donde está el type
nombres_tipos = set(pelicula [2] for pelicula in datos_peliculas)
print("Los tipos dentro de posición 2 son:", nombres_tipos)

# Crear listas vacías para cada tipo de película en minúsculas y añadir info con list comprehension
tv_mini_series_lista = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "TV Mini Series"]
movie = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "Movie"]
tv_series = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "TV Series"]
short = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "Short"]
tv_episode = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "TV Episode"]
video = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "Video"]
tv_movie = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "TV Movie"]
video_game = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "Video Game"]

print(movie)


Los tipos dentro de posición 2 son: {'Short', 'TV Series', 'Music Video', 'Video', 'TV Mini Series', 'TV Episode', 'Video Game', 'TV Movie', 'Movie'}
[('tt0143558', 'Apaföld', 'Movie', '2009', 'Drama'), ('tt0153140', 'Rózsaszín sajt', 'Movie', '2009', 'Drama'), ('tt0205380', 'Sanam Teri Kasam', 'Movie', '2009', 'Drama'), ('tt0254832', 'Twice as Dead', 'Movie', '2009', 'Drama'), ('tt0311698', 'Railed', 'Movie', '2009', 'Drama'), ('tt0327597', 'Coraline', 'Movie', '2009', 'Drama'), ('tt0344074', 'Mitsein', 'Movie', '2009', 'Drama'), ('tt0346631', 'Blood and Bone', 'Movie', '2009', 'Drama'), ('tt0361748', 'Inglourious Basterds', 'Movie', '2009', 'Drama'), ('tt0362478', 'The Box', 'Movie', '2009', 'Drama'), ('tt0288190', 'El reino de los cielos', 'Movie', '2009', 'Comedy'), ('tt0304876', 'Unter Strom', 'Movie', '2009', 'Comedy'), ('tt0312958', 'Chinango', 'Movie', '2009', 'Comedy'), ('tt0353765', 'Los muertos van deprisa', 'Movie', '2009', 'Comedy'), ('tt0365873', 'Two-Minute Heist', 'Movi

- **Datos Selenium IMDB**

In [55]:

# Iniciar el navegador
driver = webdriver.Chrome()
print("Abriendo Google")
# Maximizar la ventana del navegador
driver.maximize_window()
print("Pantalla maximizada")
sleep(3)

# Abrir página web
driver.get("https://www.imdb.com/")
print("Accediendo a la web")
sleep(3)

# Aceptar cookies
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
print("Cookies aceptadas")
sleep(5)

# Diccionario vacío en dónde se almacenarán los datos
datos_imdb = {"puntuacion":[], "direccion":[], "guionista":[], "argumento":[], "nombre_pelicula":[], "id_pelicula":[]}

# Obtiene los nombres en la posición 2 (0,1,2) que es en donde está el type
nombres_tipos = set(pelicula[2] for pelicula in datos_peliculas)
print("Los tipos dentro de posición 2 son:", nombres_tipos)

# Crear listas vacías para cada tipo de película en minúsculas y añadir info con list comprehension
#tv_mini_series_lista = [pelicula for pelicula in datos_peliculas if pelicula[2] == "TV Mini Series"]
movie = [pelicula for pelicula in datos_peliculas if pelicula[2] == "Movie"]
#tv_series = [pelicula for pelicula in datos_peliculas if pelicula[2] == "TV Series"]
short = [pelicula for pelicula in datos_peliculas if pelicula[2] == "Short"]


# Iterar sobre los IDs de las películas
id_pelicula_list = [pelicula[0] for pelicula in movie]

# Iterar sobre los IDs de las películas
for id_pelicula in id_pelicula_list:
    url_pelicula = f"https://www.imdb.com/title/{id_pelicula}/"
    
    # Navegar a la página de la película usando el ID
    driver.get(url_pelicula)
    sleep(5)

    try:
        # Extraer el nombre de la película
        titulo = driver.find_element("css selector", "h1 span").text
        datos_imdb["nombre_pelicula"].append(titulo)
        datos_imdb["id_pelicula"].append(id_pelicula)
        print(f"Titulo: {titulo}")

        # Extraer la puntuación
        try:
            puntuacion = driver.find_element("css selector", "span.sc-eb51e184-1.ljxVSS").text
            datos_imdb["puntuacion"].append(puntuacion)
            print(f"Puntuacion: {puntuacion}")
        except:
            datos_imdb["puntuacion"].append("N/A")
            print("Puntuación no encontrada")

        # Extraer la dirección
        try:
            directores = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.gxWIhN > div.sc-491663c0-6.etrlfo > div.sc-491663c0-10.iaQXlA > section > div.sc-1f50b7c-3.gLpgJQ > div > ul > li:nth-child(1) > div').text
            datos_imdb["direccion"].append(directores)
            print(f"Dirección: {directores}")

        except:
                try:
                    directores = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.ILcwq > div.sc-491663c0-6.bvzCJs > div.sc-491663c0-10.tkbDg > section > div.sc-1f50b7c-3.gLpgJQ > div > ul > li:nth-child(1) > div').text
                    datos_imdb["direccion"].append(directores)
                    print(f"Dirección: {directores}")
                except:
                    datos_imdb["direccion"].append("N/A")
                    print("Dirección no encontrada")

  

        # Extraer guionista
        try:
            guionista = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.gxWIhN > div.sc-491663c0-6.etrlfo > div.sc-491663c0-10.iaQXlA > section > div.sc-1f50b7c-3.gLpgJQ > div > ul > li:nth-child(2) > div').text
            datos_imdb["guionista"].append(guionista)
            print(f"Guionista: {guionista}")
        except:
            try:
                guionista = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.ILcwq > div.sc-491663c0-6.bvzCJs > div.sc-491663c0-10.tkbDg > section > div.sc-1f50b7c-3.gLpgJQ > div > ul > li:nth-child(2) > div').text
                datos_imdb["guionista"].append(guionista)
                print(f"Guionista: {guionista}")
            except:
                datos_imdb["guionista"].append("N/A")
                print("Guionista no encontrado")

        # Extraer argumento
        try:
            argumento = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.gxWIhN > div.sc-491663c0-6.etrlfo > div.sc-491663c0-10.iaQXlA > section > p > span.sc-2d37a7c7-2.ggeRnl').text
            datos_imdb["argumento"].append(argumento)
            print(f"Argumento: {argumento}")

           
        except:
            
            try: 
                argumento = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.ILcwq > div.sc-491663c0-6.bvzCJs > div.sc-491663c0-10.tkbDg > section > p > span.sc-2d37a7c7-2.ggeRnl').text
                datos_imdb["argumento"].append(argumento)
                print(f"Argumento: {argumento}")

            except:
                datos_imdb["argumento"].append("N/A")
                print("Argumento no encontrado")

    except:
        print(f"Error al procesar la película con ID '{id_pelicula}'")
        datos_imdb["nombre_pelicula"].append("N/A")
        datos_imdb["id_pelicula"].append(id_pelicula)
        datos_imdb["puntuacion"].append("N/A")
        datos_imdb["direccion"].append("N/A")
        datos_imdb["guionista"].append("N/A")
        datos_imdb["argumento"].append("N/A")

print(datos_imdb)
# Cerrar el navegador
driver.quit()

Abriendo Google
Pantalla maximizada
Accediendo a la web
Cookies aceptadas
Los tipos dentro de posición 2 son: {'Short', 'TV Series', 'Music Video', 'Video', 'TV Mini Series', 'TV Episode', 'Video Game', 'TV Movie', 'Movie'}
Titulo: Apaföld
Puntuacion: 5,2
Dirección: Viktor Oszkar Nagy
Guionista: Viktor Oszkar Nagy
Argumento: 
Titulo: Rózsaszín sajt
Puntuacion: 5,9
Dirección: Barnabás Tóth
Guionista: Barnabás TóthFrank RizzoDániel Hamvas
Argumento: To escape the oppressive hyperactivity of his gynecologist father, Dani, 19-year-old student at the Medical University of Budapest, travels to France to his musician uncle. A vision of adolescence and father-son-women relationship filled with humour and human touches. This movie is spoken in Hungarian and French and subtitled in English.
Titulo: Sanam Teri Kasam
Puntuacion: 4,2
Dirección: Lawrence D'Souza
Guionista: Talat Rekhi
Argumento: Se forma un triángulo amoroso cuando un hombre descubre que su mejor amigo se ha casado con su ex prometi

KeyboardInterrupt: 

- **EXTRACCION ACTORES** 

In [ ]:
#prueba actores

# Cargar el archivo CSV
movies_df = pd.read_csv('movies_last_15_years.csv', header=None, names=['codigo', 'nombre', 'tipo', 'año', 'genero'])

#Abrir el navegador
driver = webdriver.Chrome()
print("Abriendo Google")


# Maximizar la ventana del navegador
driver.maximize_window()
print("Pantalla maximizada")
sleep(3)

# Abrir página web
driver.get("https://www.imdb.com/")
print("Accediendo a la web")
sleep(3)


#COOKIES
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
print("Cookies aceptadas")
sleep(5)


# Almacenar resultados
datos_actores = {"nombre":[], "año_nacimiento":[], "Es conocido por":[], "Qué hace?":[], "premios":[]}

# Iterar sobre cada película en el DataFrame
for index, row in movies_df.iterrows():
    movie_title = row['nombre']

# Iterar sobre los IDs de las películas
for id_pelicula in id_pelicula_list:
    url_pelicula = f"https://www.imdb.com/title/{id_pelicula}/"
    
    # Navegar a la página de la película usando el ID
    driver.get(url_pelicula)
    sleep(5)


    driver.implicitly_wait(10)  # Espera implícita de 10 segundos

    # Encuentra el elemento SVG usando un selector XPath (puedes ajustar el XPath según sea necesario)
    boton_actores = driver.find_element(By.XPATH, "//svg[@class='ipc-icon ipc-icon--chevron-right']")

    # Da clic en el elemento SVG
    boton_actores.click()

    print("Hiciste clic en el elemento SVG.")
    
    nombre_actor = driver.find_elements('id', 'boton_id')
    nombre_actor.click()


- **Oscares**

In [49]:
# URL de la página de Wikipedia
url_oscar = "https://es.wikipedia.org/wiki/Premios_%C3%93scar"

# Realizar la solicitud HTTP a la página
res_url_oscar = requests.get(url_oscar)

# Verificar si la solicitud fue exitosa
if res_url_oscar.status_code == 200:
    # Parsear el contenido HTML con BeautifulSoup
    sopa_oscar = BeautifulSoup(res_url_oscar.content, "html.parser")

    # dict vacío en dónde se almacenarán los datos encontrados
    dict_datos_oscar = {"Ceremonia": [], "Mejor película": [], "Mejor director": [], "Mejor actor": [], "Mejor actriz": []}

    # Extraemos los datos de, únicamente la tabla dada.
    tabla_oscar = sopa_oscar.select_one("#mw-content-text .wikitable")
    # css selector COMPLETO: #mw-content-text > div.mw-content-ltr.mw-parser-output > table.wikitable
    # El selector se ha simplificado en caso de haber tablas anidadas. Y PORQUE CON EL COMPLETO NO NOS SALÍA ...... 
    
    # Crear una lista vacía para almacenar las tuplas
    lista_tuplas_oscar = []

    # Verificar que la tabla haya sido extraída correctamente
    if tabla_oscar:
        # Iterar sobre la tabla buscando las filas (Inspeccionando la web, vemos que las filas están en tr)
        filas = tabla_oscar.find_all('tr')
        
        # Iterando sobre las filas encontradas dentro de la tabla
        for fila in filas[1:]:  # La primera fila es el encabezdo que no necesitamos.
            
            celdas = fila.find_all('td') # Inspeccionando la web, vemos que en td, está el contenido de las celdas
            if len(celdas) >= 6: # Hay que revisar la cantidad de posiciones que tenemos
                
                # Sacar el año de la celda seleccionada en el selector wikitable
                fecha_divida = ceremonia.split() # dividir el texto
                fecha_dividida_lista = list(fecha_divida) #convirtiendo a lista
                for i in fecha_dividida_lista: # iterando por los caracteres
                    numeros = i[-4::]     #trayendo unicamente los ultimos 4 caracter que corresponden al año
                anio = int(numeros)  # conviertiendo esos 4 ultimos caracteres a int para que sea un dato numérico
                 

                #if anio >= 2000:
                # Extraer contenido de celdas por posición/ index 
                ceremonia = celdas[1].text.strip() #extraemos por index y sólo el texto y sin espacios extras
                mejor_pelicula = celdas[2].text.strip()
                mejor_director = celdas[3].text.strip()
                mejor_actor = celdas[4].text.strip()
                mejor_actriz = celdas[5].text.strip()
                
                # Añadir los datos al diccionario
                dict_datos_oscar["Ceremonia"].append(anio) #es el entero sacado antes
                dict_datos_oscar["Mejor película"].append(mejor_pelicula)
                dict_datos_oscar["Mejor director"].append(mejor_director)
                dict_datos_oscar["Mejor actor"].append(mejor_actor)
                dict_datos_oscar["Mejor actriz"].append(mejor_actriz)

                # Creamos una tupla con los datos extraídos de cada fila de la tabla y se añaden a la lista de tuplas ("lista_tuplas_oscar")
                tupla = (anio, mejor_pelicula, mejor_director, mejor_actor, mejor_actriz)
                lista_tuplas_oscar.append(tupla)


        # Convertir el diccionario en un DataFrame de pandas
        df_oscar = pd.DataFrame(dict_datos_oscar)
        # Mostrar el DataFrame
        print(df_oscar)

         # Iterar y printar cada tupla de la lista de tuplas ("lista_tuplas_oscar")
        print("_"*280)
        print("Lista de tuplas:")

        for tupla in lista_tuplas_oscar:
            print(lista_tuplas_oscar)

    else:
        print("No se ha encontrado la tabla 'wikitable'")
else:
    print(f"Error al acceder a la página: {res_url_oscar.status_code}")


    Ceremonia                                     Mejor película                                     Mejor director                                    Mejor actor                                       Mejor actriz
0        2024                                Alas (Wings) (1927)              Drama:  F. BorzageCom.:  L. Milestone                  E. Jannings(The Last Command)                              J. Gaynor(7th Heaven)
1        1929  La melodía de Broadway (The Broadway melody) (...                          F. Lloyd(The Divine Lady)                      W. Baxter(In Old Arizona)                              M. Pickford(Coquette)
2        1930  Sin novedad en el frente (All quiet on the wes...  L. MilestoneSin novedad en el frente (All quie...                            G. Arliss(Disraeli)                           N. Shearer(The Divorcee)
3        1930                                    Cimarron (1931)                                  N. Taurog(Skippy)                      L. Barr